In [1]:
from parquet_data_reader import ParquetDataReader
import pandas as pd

data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../data/document_vector.parquet')

pd.set_option('display.max_columns', None)

ValueError: Failed to read the Parquet file: [Errno 2] No such file or directory: '../data/document_vector.parquet'

In [ ]:
train_behaviors_df.head()

In [ ]:
articles_df.head()

In [ ]:
train_history_df.head()

In [ ]:
document_vectors_df.head()

In [ ]:
print('Articles columns and data types:')
print(articles_df.dtypes)

print('\nTrain behaviors columns and data types:')
print(train_behaviors_df.dtypes)

print('\nTrain history columns and data types:')
print(train_history_df.dtypes)

print('\nDocument vectors columns and data types:')
print(document_vectors_df.dtypes)


In [ ]:
# Check for missing values
print('Missing values in Articles | Total number of rows: ', len(articles_df))
print(articles_df.isnull().sum())

print('\nMissing values in Train behaviors: | Total number of rows: ', len(train_behaviors_df))
print(train_behaviors_df.isnull().sum())

print('\nMissing values in Train history:| Total number of rows: ', len(train_history_df))
print(train_history_df.isnull().sum())

print('\nMissing values in Document vectors: | Total number of rows: ', len(document_vectors_df))
print(document_vectors_df.isnull().sum())
